In [ ]:
import anndata
import pandas as pd
from pathlib import Path

llava_annotations=pd.read_csv(snakemake.input.cellwhisperer_labels)


In [ ]:
from openai import OpenAI

system_message = snakemake.params.request

client = OpenAI(api_key=snakemake.params.openai_api_key)

In [ ]:
assert len(llava_annotations) <= snakemake.params.max_num_clusters, f"too many clusters in dataset ({num_clusters}). Stopping to prevent high GPT-4 cost"

In [ ]:
results = {}
for idx, row in llava_annotations.iterrows():
    request = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": row["cluster_annotations"]},
    ]
    if not isinstance(row["cluster_annotations"], str):
        response = "No label"
    else:
        chat_completion = client.chat.completions.create(
            messages=request,
            model="gpt-4-turbo-preview",
            max_tokens=20,
            temperature=0,  # type: ignore [reportUndefinedVariable]
        )
        response = chat_completion.choices[0].message.content

    print(response)
    results[idx] = response

In [ ]:
llava_annotations["curated_labels"] = results

In [ ]:
llava_annotations.to_csv(snakemake.output.curated_labels, index=False)